In [1]:
# load some libraries
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from tqdm import tqdm
from sklearn.metrics import accuracy_score


import numpy as np
from sklearn.model_selection import train_test_split

from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.ensemble import BaggingClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.pipeline import Pipeline

import joblib

import predictgame as pg
from worldcuptransformers import CustomTransformer

import random
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore", UserWarning)


from datetime import datetime
import logging
dateTimeObj = datetime.now()
timestampStr = dateTimeObj.strftime("%Y%m%d%H%M%S")
logging.basicConfig(filename=f'compute_wc_log/compute-{timestampStr}.log',  level=logging.DEBUG,
    format='%(message)s', datefmt='%m/%d/%Y %I:%M:%S %p')
timestampStr


ModuleNotFoundError: No module named 'predictgame'

In [2]:
result_worldcup = []
number_of_wc_to_play = 1

In [3]:
FOLDER = 'dataset'
FILENAME = 'wc_2018.csv'
PATH = f'{FOLDER}/{FILENAME}'
logging.info(f'compute World Cup {PATH}')

for wc_number in tqdm(range(number_of_wc_to_play)):
    # define some constant

    wc = pd.read_csv(f'{PATH}')
    wc.drop('Unnamed: 0', axis=1, inplace=True)
    wc['matchday'] = pd.to_datetime(wc['matchday'], format='%Y-%m-%d')
    wc.sort_values('matchday', inplace=True)
    # define features
    features = ['1N2_1','1N2_N','1N2_2','home_team_rank_FIFA','home_team_trend','home_team_odd_trend','corr_score_home',
                                        'away_team_rank_FIFA','away_team_trend','away_team_odd_trend','corr_score_away']

    for idx, r in wc.iterrows():
        diff_fifa=abs(r.home_team_rank_FIFA - r.away_team_rank_FIFA)
        o1 = r['1N2_1']*random.uniform(0.805, 1.85) #* diff_fifa 
        oN = r['1N2_N']*random.uniform(0.505, 0.135) #* diff_fifa 
        o2 = r['1N2_2']*random.uniform(0.705, 1.65) #* diff_fifa 
        
        wc.loc[idx,'1N2_1']=o1
        wc.loc[idx,'1N2_N']=oN
        wc.loc[idx,'1N2_2']=o2

    # wc.loc[idx,features]=random.random()
        
        
        
        t = sorted([o1,oN,o2])
        wc.loc[idx,'odd_min']=t[1]
        wc.loc[idx,'odd_max']=t[2]
        wc.loc[idx,'lowest_odd']=t[0]

    wc.shape
    useless = [c for c in wc.columns if 'Unnamed' in c]
    try:
        wc.drop(useless, axis=1, inplace=True)
    except:
        pass



    ## Play the 48 matchs of the groupe stage phase and check

    #scaler = StandardScaler()
    #scaler.fit(wc[features])

    res=[]
    for m in range(0,48):
        payout = wc.loc[m,features].values.reshape(1,-1)


        # call the predictGame class and predict method
        feat = ['1N2_1','1N2_N','1N2_2','home_team_rank_FIFA','away_team_rank_FIFA','season','home_team','away_team']
        feat = ['1N2_1','1N2_N','1N2_2','home_team_rank_FIFA','away_team_rank_FIFA','season']
        play=pd.DataFrame(wc.iloc[m,:]).T
        #print(play)
        #load your model for further usage
        #play[['corr_score_home', 'corr_score_away']]=0

        g = pg.PredictGame(play)
        p = g.predict()
        t=p[0][0][7]
        #print(f'Prediction= {p}')
        
        result_proba_1N2=[]
        result_proba_12=[]
        rem = f'GAME {m+1} - {play.home_team.values[0]} Vs. {play.away_team.values[0]} - Prediction={p[0][0][4]}/{p[0][0][5]} = {np.int(p[0][0][8][0])}:{np.int(p[0][0][8][1])}'
        logging.info(rem)




            
        #res.append([wc.loc[m,'matchday'],wc.loc[m,'home_team'],wc.loc[m,'away_team'],wc.loc[m,'correct_score'],pred1, result, result_proba_1N2, result_proba_12,score.astype(int)])    
        res.append([wc.loc[m,'matchday'],wc.loc[m,'home_team'],wc.loc[m,'away_team'],wc.loc[m,'correct_score'],
                        p[0][0][4], p[0][0][5], result_proba_1N2, result_proba_12,p[0][0][8]])    
        #print(p)
        wc.loc[m,'target_predicted']=p[0][0][4]  
        #wc.loc[m,'score_predicted']=score  

        
    #len(res)   
    pred=pd.DataFrame(res).dropna()
    pred.columns = ['matchday','home_team','away_team','correct_score','target','target_predict','target_proba_1','target_proba_z','score_predict']
    for idx, r in pred.iterrows():
        #pred.loc[idx,'score_pred']=f'{str(r.score_predict[0][0].astype(int))}:{str(r.score_predict[0][1].astype(int))}'
        pred.loc[idx,'score_pred']=f'{r.score_predict[0]}:{r.score_predict[1]}'

        #print(r.score_predict)
    
    for idx, r in pred.iterrows():
        #h_score = r.correct_score.split(':')[0]
        #a_score = r.correct_score.split(':')[1]
        #print(r.score_predict)
        h_score = r.score_predict[0]
        a_score = r.score_predict[1]

        pred.loc[idx,'home_score']=int(h_score)
        pred.loc[idx,'away_score']=int(a_score)
        if h_score > a_score:
            pred.loc[idx,'h_points']=3
            pred.loc[idx,'a_points']=0
        elif h_score == a_score:
            pred.loc[idx,'h_points']=1
            pred.loc[idx,'a_points']=1
        else:
            pred.loc[idx,'h_points']=0
            pred.loc[idx,'a_points']=3

            
        
        if h_score == a_score:
            pred.loc[idx,'game_bet']='N'
        elif h_score > a_score:
            pred.loc[idx,'game_bet']='1'   
        else:
            pred.loc[idx,'game_bet']='2' 
            
        h_score = r.score_pred.split(':')[0]
        a_score = r.score_pred.split(':')[1]
        h_score = r.correct_score.split(':')[0]
        a_score = r.correct_score.split(':')[1]
        if h_score == a_score:
            pred.loc[idx,'game_pred']='N'
        elif h_score > a_score:
            pred.loc[idx,'game_pred']='1'   
        else:
            pred.loc[idx,'game_pred']='2' 
        
    pred.drop('score_predict', axis=1).sort_values('matchday')

    # World Cup Table
    #The structure of the world cup is stored in a json file
    WC_TABLE = 'wc_2018_table.json'
    import pandas as pd
    import json
    import os


    # Reading the json as a dict
    with open('dataset/wc_2018_table.json') as json_data:
        data = json.load(json_data)
        
    wc_table = pd.DataFrame.from_dict(data['stage'], orient='index').T.reset_index()
    #print(wc_table)
    table = pd.DataFrame()
    for idx, r in wc_table.iterrows():
        teams = r.groups
        group = r['index']
        for team in teams:
            # look in predict table for point and score
            h=pred.query('home_team == @team')[['home_score','away_score','h_points']]
            h.columns=['for','against','points']
            a=pred.query('away_team == @team')[['away_score','home_score','a_points']]
            a.columns=['for','against','points']

            frames = [a,h]
            t=pd.DataFrame(pd.concat(frames).sum()).T
            t['team']=team      
            t['group']=group
            
            frames = [table, t]
            table = pd.concat(frames)  
            


    cols = ['group','team','for','against','diff','points']
    table['diff']= table['for']-table['against']
    table=table[cols].sort_values(['group','points','diff','for','against'], ascending=[True,False,False,False,True])
    logging.info('')
    logging.info('Tables after Group Stage')
    logging.info('---------------------------------------')
    logging.info(table)


    first_in_group = table.groupby(['group']).nth(0).copy()
    for idx, r in first_in_group.iterrows():
        grp = idx.split('-')
        code = f"1{grp[1].upper()}"
        first_in_group.loc[idx,'code']=code


    second_in_group = table.groupby(['group']).nth(1).copy()
    for idx, r in second_in_group.iterrows():
        grp = idx.split('-')
        code = f"2{grp[1].upper()}"
        second_in_group.loc[idx,'code']=code

        
        
        
    frames = [first_in_group,second_in_group]
    logging.info('')
    logging.info('1st After Group Stage')
    logging.info('---------------------------------------')
    logging.info(first_in_group)

    logging.info('')
    logging.info('2nd After Group Stage')
    logging.info('---------------------------------------')
    logging.info(second_in_group)

    
    
    next_game = pd.concat(frames)
    next_game
    ko16 = pd.DataFrame.from_dict(data['knockouts'], orient='index').T.reset_index()
    ko16
    games16 = []
    for idx, r in ko16.iterrows():
        g1 = r['knockout-16'][0]
        g2 = r['knockout-16'][1]
        
        t1 = g1[0]
        t2 = g1[1]
        
        games16.append([next_game.query('code == @t1').team.values[0],next_game.query('code == @t2').team.values[0]])
        
        t1 = g2[0]
        t2 = g2[1]
        
        games16.append([next_game.query('code == @t1').team.values[0],next_game.query('code == @t2').team.values[0]])
        
        
            
    games16
    # all time games
    alg = pd.read_csv(f'{FOLDER}/training_dataset.csv')
    alg.shape
    # if no data are found so this function will create a dummy entries with teams
    # the scope of data is based on feat which is the collection of data need

    def define_unknown_game(home, away):
        """_summary_
        Look inside the training_dataset for both teams games and fifa ranking and the collect
        same pattern game

        Args:
            home (_type_): _description_
            away (_type_): _description_
        """
        #print(home, away)
        # get a list of ranking FIFA
        rk = pd.read_csv('dataset/fifa_ranking-2021-05-27.csv')
        rk['rank_date']=pd.to_datetime(rk['rank_date'])
        rk['rank_year']=rk.rank_date.dt.year
        rk = rk.query('rank_year == 2018').copy()
        rk = rk.groupby('country_full').mean()

        ts = pd.read_csv('dataset/training_dataset.csv')
        ts['FIFA_diff']=abs(ts['home_team_rank_FIFA']-ts['away_team_rank_FIFA'])
        
        rk= rk.query('country_full == @home or country_full == @away')
        try:
            home_rank = rk.query('country_full == @home')['rank'][0]
            away_rank = rk.query('country_full == @away')['rank'][0]
        except:
            home_rank=99
            away_rank=99

        diff = abs(home_rank - away_rank)
        
        subset_ts = ts.query('(FIFA_diff > @diff*0.85 and FIFA_diff < @diff*1.15)')
        subset_ts= pd.DataFrame(subset_ts.mean()).T
        subset_ts['matchday']=None
        subset_ts['home_team']=home
        subset_ts['away_team']=away
        subset_ts['correct_score']='0:0'
        try:
            return rk.query('country_full == @home')['rank'][0],rk.query('country_full == @away')['rank'][0], subset_ts
        except:
            return 99,99, subset_ts


        
    rh, ra, ts = define_unknown_game('Belgium','Senegal')
    rh, ra


    games16
    pko = []
    for game in games16:
        t1 = game[0]
        t2 = game[1]
        g = alg.query('(home_team == @t1) and (away_team == @t2)').index
        #print(g, len(g))
        # if len(g) >1 then call PredictGame with data
        if len(g) > 0:
            #print(pd.DataFrame(alg.iloc[g[0],:]).T)
            play=pd.DataFrame(alg.iloc[g[0],:]).T.fillna(0)
            #play[['corr_score_home', 'corr_score_away']]=0
            g = pg.PredictGame(play, True)
            p = g.predict()
            t=p[0][0][7]
            pko.append([p, t1,t2])
        else:
            rh, ra, ts = define_unknown_game(t1,t2)
            play = ts.fillna(0)
            g = pg.PredictGame(play, True)
            p = g.predict()
            t=p[0][0][7]
            pko.append([p, t1,t2])

                
        
    pko[0]
    del_col = [x for x in play.columns.tolist() if '_x' in x or '_y' in x]
    try:
        play.drop(['Unnamed: 0','Unnamed: 0.1','Unnamed: 0.1.1'], axis=1, inplace=True)
        play.drop(del_col, axis=1, inplace=True)

    except:
        pass

    play

    p=pko[0]
    len(p)
    len(p[0])
    ko16_res=[]
    g_number = 0
    for k in range(0,len(pko)):
        g_number += 1
        #print((pko[k][0][0][0][8]), pko[k][1], pko[k][2])
        ko16_res.append([pko[k][0][0][0][8][0],pko[k][0][0][0][8][1],pko[k][1],pko[k][2], g_number])
    ko16_res

    ko16_res
    q_final = []
    for k in ko16_res:
        if k[0]>k[1]:
            #print(k[2])
            q_final.append(k[2])
        else:
            #print(k[3])  
            q_final.append(k[3])  
    hq=q_final[::2]
    for h in hq:
        q_final.remove(h)

    q_final_game = []    
    for x in range(0,4):
        q_final_game.append([q_final[x], hq[x]])
        
    q_final_game

        
    pko = []
    for game in q_final_game:
        t1 = game[0]
        t2 = game[1]
        g = alg.query('(home_team == @t1) and (away_team == @t2)').index
        #print(g, len(g))
        # if len(g) >1 then call PredictGame with data
        if len(g) > 0:
            #print(pd.DataFrame(alg.iloc[g[0],:]).T)
            play=pd.DataFrame(alg.iloc[g[0],:]).T
            #play[['corr_score_home', 'corr_score_away']]=0
            g = pg.PredictGame(play, True)
            p = g.predict()
            t=p[0][0][7]
            pko.append([p, t1,t2])
        else:
            rh, ra, ts = define_unknown_game(t1,t2)
            play = ts.fillna(0)
            g = pg.PredictGame(play, True)
            p = g.predict()
            t=p[0][0][7]
            pko.append([p, t1,t2])



    ko08_res=[]
    g_number = 0
    for k in range(0,len(pko)):
        g_number += 1
        #print((pko[k][0][0][0][8]), pko[k][1], pko[k][2])
        ko08_res.append([pko[k][0][0][0][8][0],pko[k][0][0][0][8][1],pko[k][1],pko[k][2], g_number])
    ko08_res
    q_final = []
    for k in ko08_res:
        if k[0]>k[1]:
            #print(k[2])
            q_final.append(k[2])
        else:
            #print(k[3])  
            q_final.append(k[3])  
    hq=q_final[::2]
    for h in hq:
        q_final.remove(h)

    q_final_game = []    
    for x in range(0,2):
        q_final_game.append([q_final[x], hq[x]])
        
    q_final_game
    pko = []
    for game in q_final_game:
        t1 = game[0]
        t2 = game[1]
        g = alg.query('(home_team == @t1) and (away_team == @t2)').index
        #print(g, len(g))
        # if len(g) >1 then call PredictGame with data
        if len(g) > 0:
            #print(pd.DataFrame(alg.iloc[g[0],:]).T)
            play=pd.DataFrame(alg.iloc[g[0],:]).T
            #play[['corr_score_home', 'corr_score_away']]=0
            g = pg.PredictGame(play, True)
            p = g.predict()
            t=p[0][0][7]
            pko.append([p, t1,t2])
        else:
            rh, ra, ts = define_unknown_game(t1,t2)
            play = ts.fillna(0)
            g = pg.PredictGame(play, True)
            p = g.predict()
            t=p[0][0][7]
            pko.append([p, t1,t2])



    ko04_res=[]
    g_number = 0
    for k in range(0,len(pko)):
        g_number += 1
        #print((pko[k][0][0][0][8]), pko[k][1], pko[k][2])
        ko04_res.append([pko[k][0][0][0][8][0],pko[k][0][0][0][8][1],pko[k][1],pko[k][2], g_number])
    ko04_res
    q_final = []
    for k in ko04_res:
        if k[0]>k[1]:
            #print(k[2])
            q_final.append(k[2])
        else:
            #print(k[3])  
            q_final.append(k[3])  
    hq=q_final[::2]
    for h in hq:
        q_final.remove(h)

    q_final_game = []    
    for x in range(0,1):
        q_final_game.append([q_final[x], hq[x]])
        
    q_final_game
    pko = []
    for game in q_final_game:
        t1 = game[0]
        t2 = game[1]
        g = alg.query('(home_team == @t1) and (away_team == @t2)').index
        #print(g, len(g))
        # if len(g) >1 then call PredictGame with data
        if len(g) > 0:
            #print(pd.DataFrame(alg.iloc[g[0],:]).T)
            play=pd.DataFrame(alg.iloc[g[0],:]).T
            #play[['corr_score_home', 'corr_score_away']]=0
            g = pg.PredictGame(play, True)
            p = g.predict()
            t=p[0][0][7]
            pko.append([p, t1,t2])
        else:
            rh, ra, ts = define_unknown_game(t1,t2)
            play = ts.fillna(0)
            g = pg.PredictGame(play, True)
            p = g.predict()
            t=p[0][0][7]
            pko.append([p, t1,t2])



    ko02_res=[]
    g_number = 0
    for k in range(0,len(pko)):
        g_number += 1
        #print((pko[k][0][0][0][8]), pko[k][1], pko[k][2])
        ko02_res.append([pko[k][0][0][0][8][0],pko[k][0][0][0][8][1],pko[k][1],pko[k][2], g_number])
    #print(ko02_res)
    result_worldcup.append(ko02_res[0])

100%|██████████| 1/1 [00:03<00:00,  3.27s/it]


In [4]:
wc_glb=pd.DataFrame(result_worldcup, 
columns=['ht_score','at_score','home_team',
'away_team','check'])
for idx, r in wc_glb.iterrows():
    if r.ht_score > r.at_score:
        wc_glb.loc[idx,'winner']=r.home_team
    else:
        wc_glb.loc[idx,'winner']=r.away_team
        
wc_glb.value_counts('winner', normalize=True)

winner
Germany    1.0
dtype: float64

In [5]:
play.home_team.values[0]

'Germany'